In [156]:
import os
import io
import time
import datetime
import pandas as pd
import xlsxwriter
import numpy as np
import requests
import unidecode

ModuleNotFoundError: No module named 'unidecode'

In [83]:
filepath_data = r"C:\Users\gille\Desktop\DESU Data Science\070 - Machine Learning\010 - Datasets"+chr(92)
filepath_processed = r"C:\Users\gille\Desktop\DESU Data Science\070 - Machine Learning\010 - Datasets"+chr(92)
filepath_interim = r"C:\Users\gille\Desktop\DESU Data Science\070 - Machine Learning\010 - Datasets"+chr(92)
file_xls_analyse = "Analyse_dataframe_ges.xlsx"

In [84]:
def debug(message=" ") :
    ts = time.time()
    ts = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
    print ("{} Debug -> ".format(ts)+str(message))

In [85]:
def df_get (filepath) :
    list_df=list()
    files = [f for f in os.listdir(filepath_data) if f.endswith(".csv")]
    
    for i, file in enumerate(files):
        debug("Agregation du fichier :" + file)
        list_df.append(pd.read_csv(filepath_data + chr(92) + file, delimiter=',', low_memory=False,
                                   decimal=".",
                                   dtype={"Valeur": np.float64},
                                   skiprows=10
                                  ))

        debug("Nombre de ligne chargées :" + str(len(list_df[i])))
        debug()

    debug("Fusion des Dataframes")
    df = pd.concat(list_df)
    debug()

    debug("Netoyage des headers de colonne")
    df.rename(columns=lambda x: x.replace(' ', '_').replace('/', '_'), inplace=True)
    debug()
    return df

# Main

In [ ]:
df_ges=df_get(filepath_data)

In [87]:
df_ges.head()

,Année,Entité_administrative,Activité,Energie,Variable,Valeur,Unite
0,2007,ABRIÈS (05),Tous secteurs,Toutes énergies,prg100.3ges,3509986.4,kg
1,2007,AIGLUN (04),Tous secteurs,Toutes énergies,prg100.3ges,9428086.6,kg
2,2007,AIGLUN (06),Tous secteurs,Toutes énergies,prg100.3ges,303085.4,kg
3,2007,AIGUILLES (05),Tous secteurs,Toutes énergies,prg100.3ges,3517602.4,kg
4,2007,AIGUINES (83),Tous secteurs,Toutes énergies,prg100.3ges,7928209.2,kg


In [88]:
df_ges["Valeur"] = df_ges["Valeur"]/1000

In [128]:
df_ges.isna().sum()

Année                      0
Entité_administrative      0
Activité                   0
Energie                    0
Variable                   0
Valeur                   418
Unite                      0
Ville                      0
dept                       0
dtype: int64

In [89]:
df_ges.head()

,Année,Entité_administrative,Activité,Energie,Variable,Valeur,Unite
0,2007,ABRIÈS (05),Tous secteurs,Toutes énergies,prg100.3ges,3509.9864,kg
1,2007,AIGLUN (04),Tous secteurs,Toutes énergies,prg100.3ges,9428.0866,kg
2,2007,AIGLUN (06),Tous secteurs,Toutes énergies,prg100.3ges,303.0854,kg
3,2007,AIGUILLES (05),Tous secteurs,Toutes énergies,prg100.3ges,3517.6024,kg
4,2007,AIGUINES (83),Tous secteurs,Toutes énergies,prg100.3ges,7928.2092,kg


In [90]:
df_ges[["Ville",'dept']]=df_ges["Entité_administrative"].str.rsplit(n=1,expand=True)

In [96]:
df_ges.head()

,Année,Entité_administrative,Activité,Energie,Variable,Valeur,Unite,Ville,dept
0,2007,ABRIÈS (05),Tous secteurs,Toutes énergies,prg100.3ges,3509.9864,kg,ABRIÈS,05
1,2007,AIGLUN (04),Tous secteurs,Toutes énergies,prg100.3ges,9428.0866,kg,AIGLUN,04
2,2007,AIGLUN (06),Tous secteurs,Toutes énergies,prg100.3ges,303.0854,kg,AIGLUN,06
3,2007,AIGUILLES (05),Tous secteurs,Toutes énergies,prg100.3ges,3517.6024,kg,AIGUILLES,05
4,2007,AIGUINES (83),Tous secteurs,Toutes énergies,prg100.3ges,7928.2092,kg,AIGUINES,83


In [95]:
df_ges["dept"]=df_ges["dept"].str.replace("(","").str.replace(")","")

In [137]:
pop_dtypes = { 'Code région': str, 'Nom de la région': str, 'Code département': str, 'Code arrondissement': str, 'Code canton': str, 'Code commune': str, 'Nom de la commune': str, 'Population municipale': int, 'Population comptée à part': int, 'Population totale': int}

In [139]:
df_communes=pd.read_excel (r"C:\Users\gille\Desktop\DESU Data Science\070 - Machine Learning\010 - Datasets\ensemble.xls", 
                    sheet_name="Communes",skiprows=7,dtype=pop_dtypes)

In [146]:
df_communes.dtypes

Code région                  object
Nom de la région             object
Code département             object
Code arrondissement          object
Code canton                  object
Code commune                 object
Nom de la commune            object
Population municipale         int32
Population comptée à part     int32
Population totale             int32
dtype: object

In [ ]:
unidecode

In [151]:
df_final = pd.merge(df_ges,
                    df_communes,
                    left_on=['Ville','dept'],
                    right_on=['Nom de la commune','Code département'],
                    how = 'left')

In [152]:
df_final

,Année,Entité_administrative,Activité,Energie,Variable,Valeur,Unite,Ville,dept,Code région,Nom de la région,Code département,Code arrondissement,Code canton,Code commune,Nom de la commune,Population municipale,Population comptée à part,Population totale
0,2007,ABRIÈS (05),Tous secteurs,Toutes énergies,prg100.3ges,3509.9864,kg,ABRIÈS,05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007,AIGLUN (04),Tous secteurs,Toutes énergies,prg100.3ges,9428.0866,kg,AIGLUN,04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007,AIGLUN (06),Tous secteurs,Toutes énergies,prg100.3ges,303.0854,kg,AIGLUN,06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007,AIGUILLES (05),Tous secteurs,Toutes énergies,prg100.3ges,3517.6024,kg,AIGUILLES,05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007,AIGUINES (83),Tous secteurs,Toutes énergies,prg100.3ges,7928.2092,kg,AIGUINES,83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7049,2016,VITROLLES (05),Tous secteurs,Toutes énergies,prg100.3ges,18008.4144,kg,VITROLLES,05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7050,2016,VITROLLES (13),Tous secteurs,Toutes énergies,prg100.3ges,234387.7914,kg,VITROLLES,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7051,2016,VITROLLES-EN-LUBÉRON (84),Tous secteurs,Toutes énergies,prg100.3ges,4312.5736,kg,VITROLLES-EN-LUBÉRON,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7052,2016,VOLONNE (04),Tous secteurs,Toutes énergies,prg100.3ges,3656.8211,kg,VOLONNE,04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
